# Attack the generative model
* Base attacks on the paper https://arxiv.org/pdf/1805.09190v3.pdf

Use attacks:
1. Gaussian Noise (random noise)
2. Salt and pepper noise (random noise)
3. Boundary Attack (non-gradient based)
4. Poinwise Attack (non-gradient based)
6. BIM/ Projected Gradient descent (gradient based)
7. FGSM (gradient based)
8. Latent Descent (novel attack for generative model)

How exactly to use these gradient based attacks?

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Nicer way to import the module?
sys.path.append(str(Path.cwd().parent))
from utils.loading import load_net
from utils.display import torch_to_np, show_imgs
from models.generative_classify import optimize_latent_cvae, gen_classify_cvae, optimize_latent, gen_classify

from models.gen_classifiers import GenerativeCVAE, GenerativeVAE


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torch.nn import functional as F
import torch.optim as optim

import foolbox
import json
from PIL import Image

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.current_device())

BASE_PATH = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/models')

cuda:1
0


## First Make VAE classifier as a pytorch model

### Test the generative classifier as a pytorch model

In [2]:
files_df_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/files_df.pkl'
with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)

img_path = files_df['val'].iloc[0]['path']
label = files_df['val'].iloc[0]['class']
transform = transforms.Compose([
                                transforms.Resize(32),
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])
tensor_img = transform(Image.open(img_path)).unsqueeze(0).to(device)
labels = list(range(10))

#### VAE

In [4]:
with torch.cuda.device(1):
    save_path = Path('/media/rene/data/adv_gen/MNIST/mnist_normal/models/VAE-1_16_32_64-16-MNIST/')
    model_dict = {}
    for label in labels:
        model_name = 'VAE-1_16_32_64-16-MNIST_label_'+str(label)+'_model_best.pth.tar'
        model_loc = save_path / model_name
        model_dict[label] = load_net(model_loc).to(device).eval()
    
    gen_model = GenerativeVAE(model_dict=model_dict, labels=labels, latent_size=16, device=device)
    pred = np.argmax(gen_model(tensor_img, iterations=50, num_times=100).cpu().numpy())
        
print(f'predicted_label: {pred}, True Label: {label}')

predicted_label: 6, True Label: 9


#### Conditional VAE Test

In [3]:
with torch.cuda.device(1):
    model_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/models/CVAE-1_16_32_64-16-MNIST-10_model_best.pth.tar'
    model = load_net(model_loc).to(device).eval()
    gen_model = GenerativeCVAE(model=model, labels=labels, latent_size=16, device=device)
    pred = np.argmax(gen_model(tensor_img, iterations=50, num_times=100).cpu().numpy())
        
print(f'predicted_label: {pred}, True Label: {label}')

results.size(), torch.Size([100, 10])
results(min):, tensor([618.2889, 625.5640, 612.2725, 623.6194, 558.3864, 584.6205, 664.2961,
        336.0796, 606.4223, 503.3734], device='cuda:1')
results.size(), torch.Size([10])
predictions:, tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000], device='cuda:1')
predicted_label: 6, True Label: 7.0


In [2]:
# import torch
# from torch import nn
# from torch.autograd import Variable
# from torchvision import transforms
# from torch.nn import functional as F
# import torch.optim as optim


# class GenerativeCVAE(nn.Module):
#     """
#     ??? ONLY USE BATCH SIZE OF 1 - Remove all the numpy stuff for results
#     ??? Not doing classification the same way as with eq.4 https://arxiv.org/pdf/1805.09190v3.pdf
#     Classify using a conditional VAE.
    
#     Do gradient descent to update using each class to reconstruct the image.
#     Initialized latent as random normal, use adam
#     labels is a list of labels
#     """
#     def __init__(self, model, labels, latent_size, device):
#         super(GenerativeCVAE, self).__init__()
#         self.labels = labels
#         self.latent_size = latent_size
#         self.model = model.eval()
#         for p in self.model.parameters():
#             p.requires_grad=False
#         self.device = device

#     def forward(self, img, iterations=50, num_times=100, KLD_weight=1, eta=1, info=False): # add in the init?
#         lr = .001
# #         results = {}
# #         for label in self.labels:
# #             results[label] = []
# #         results = np.repeat(1000000000, num_times*len(self.labels), (num_times, len(self.labels))) # no batch size
#         results = np.tile(1000000000, (num_times, len(self.labels))) # no batch size

#         results = torch.from_numpy(results).float().to(self.device)

#         for trial in range(num_times):
#             rand_mu = np.random.normal(0,1, (1, self.latent_size))
#             rand_logvar = np.random.normal(0,1, (1, self.latent_size))
#             mu = torch.from_numpy(rand_mu).float().to(self.device)
#             logvar = torch.from_numpy(rand_logvar).float().to(self.device)

#             for label in self.labels:                
#                 tensor_label = torch.from_numpy(np.array(label)).unsqueeze(0).type(torch.LongTensor).to(self.device)
#                 optimizer = optim.Adam([mu, logvar], lr=lr)

#                 for i in range(iterations):
#                     z = self.model.reparameterize(mu, logvar, deterministic=True)
#                     recon_x = self.model.decode(z, tensor_label)
#                     output = (recon_x, mu, logvar) # put it in format for the loss
#                     loss = self.model.loss(output, img, KLD_weight=KLD_weight, single_batch=True)
                    
#                     optimizer.zero_grad()
#                     loss.backward()
#                     optimizer.step()
                    
#                 results[trial, label] = loss.item()

# #                 results.scatter_(dim=0, index = torch.tensor([[0, 1, 2, 0, 0], [2, 0, 0, 1, 2]]), src=loss.item())
# #                 results[trial, label.item()].append(loss.item())
# #         mins = {k:np.min(v) for (k,v) in results.items()}
# #         predicted_label = min(mins, key=mins.get)
# #         predictions = torch.exp(results)/torch.sum(torch.exp(results))
#         results = torch.min(results, 0)[0]
#         predictions = F.softmax(results)
#         predictions = predictions.unsqueeze(0) # add the batch dim

#         if info:
#             predictions, results
#         return predictions

# class GenerativeVAE(nn.Module):
#     """ Classify using a set of VAEs
    
#     ??? ONLY USE BATCH SIZE OF 1 - Remove all the numpy stuff for results
#     ??? Not doing classification the same way as with eq.4 https://arxiv.org/pdf/1805.09190v3.pdf
#     Do gradient descent to update using each class to reconstruct the image.
#     Initialized latent as random normal, use adam
#     labels is a list of each label
#     """
#     def __init__(self, model_dict, labels, latent_size, device):
#         super(GenerativeVAE, self).__init__()
#         self.labels = labels
#         self.model_dict = model_dict
#         self.latent_size = latent_size
#         self.device = device

#     def forward(self, img, iterations=50, num_times=100,KLD_weight=1, info=False): # add in the init?
#         lr = .001
#         results = np.tile(1000000000, (num_times, len(self.model_dict.keys()))) # no batch size
#         results = torch.from_numpy(results).float().to(self.device)

#         for label, model in model_dict.items():
#             model = model.eval()
#             for p in model.parameters():
#                 p.requires_grad=False

#         for trial in range(num_times):
#             rand_mu = np.random.normal(0,1, (1, self.latent_size))
#             rand_logvar = np.random.normal(0,1, (1, self.latent_size))
#             mu = torch.from_numpy(rand_mu).float().to(self.device)
#             logvar = torch.from_numpy(rand_logvar).float().to(self.device)
            
#             for label, model in model_dict.items():
#                 tensor_label = torch.from_numpy(np.array(label)).unsqueeze(0).type(torch.LongTensor).to(self.device)
#                 optimizer = optim.Adam([mu, logvar], lr=lr)

#                 for i in range(iterations):
#                     z = model.reparameterize(mu, logvar)
#                     recon_x = model.decode(z)
#                     output = (recon_x, mu, logvar) # put it in format for the loss
#                     loss = model.loss(output, img, KLD_weight=KLD_weight)
#                     optimizer.zero_grad()
#                     loss.backward()
#                     optimizer.step()
#                 results[trial, label] = loss.item()
        
#         results = torch.min(results, 0)[0]
#         predictions = F.softmax(results)
#         predictions = predictions.unsqueeze(0) # add the batch dim

#         if info:
#             predictions, results
#         return predictions

## Simple attacks - Gaussian and Salt and Pepper Noise
* Add this noise to the imnage until is is misclassified.
* i.e. add noise to image until the loss of reconstruction is higher than the next highest class

In [3]:
from foolbox.attacks import AdditiveGaussianNoiseAttack
from models.gen_classifiers import GenerativeCVAE


num_workers = 0 ## ??? what?
batch_size = 1

num_classes = 10
labels = list(range(num_classes))

files_df_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/files_df.pkl'
with open(files_df_loc, 'rb') as f:
    files_df = pickle.load(f)

img_path = files_df['val'].iloc[4]['path']
transform = transforms.Compose([
                                transforms.Resize(32),
                                transforms.ToTensor(),
#                                 transforms.Normalize((0.1307,), (0.3081,))
                                ])
img = transform(Image.open(img_path)).unsqueeze(0).to(device)
label = files_df['val'].iloc[4]['class']

with torch.cuda.device(1):
    model_loc = '/media/rene/data/adv_gen/MNIST/mnist_normal/models/CVAE-1_16_32_64-16-MNIST-10_model_best.pth.tar'
    model = load_net(model_loc).to(device).eval()
    gen_model = GenerativeCVAE(model=model, labels=labels, latent_size=16, device=device).eval()
    predicted_label = gen_model(img, iterations=50, num_times=100, info=False)
    print('predicted_label', predicted_label)

RuntimeError: input and target shapes do not match: input [1 x 32 x 32], target [1 x 1 x 32 x 32] at /pytorch/aten/src/THCUNN/generic/MSECriterion.cu:12

In [5]:
from models.cifar import PreActResNet
from utils.loading import load_net_cifar
model_loc = '/media/rene/data/adv_consistency/MNIST/models/preact_resnet-18_model_best.pth.tar'
model = load_net_cifar(model_loc)

with torch.cuda.device(1):
    mean = np.array([0.1307]).reshape((1, 1, 1))
    std = np.array([0.3081]).reshape((1, 1, 1))
    
    fmodel = foolbox.models.PyTorchModel(model, bounds=(0, 1), preprocessing=(.1307, .3081), 
                                         num_classes=num_classes, device=device)
    attack = foolbox.attacks.AdditiveGaussianNoiseAttack(fmodel)
    
    print('img.size()', img.size())
    print('torch.min(img)', torch.min(img))
    print('torch.max(img)', torch.max(img))
    print('type(label)', type(label))
    print('label', label)

    np_img = np.expand_dims(np.squeeze(img.cpu().numpy()), 0) #/ 255.
#     np_img = np.squeeze(img.cpu().numpy())#/ 255.



    np_label = int(label)#int(label.numpy()[0])
    print('np_img.shape', np_img.shape)
    print('np.min(np_img)', np.min(np_img))
    print('np.max(np_img)', np.max(np_img))
    print('np_label', np_label)

    adv_img = attack(np_img, np_label)

Loading model_file preact_resnet-18_model_best.pth.tar
img.size() torch.Size([1, 1, 32, 32])
torch.min(img) tensor(0., device='cuda:1')
torch.max(img) tensor(0.9922, device='cuda:1')
type(label) <class 'numpy.float64'>
label 4.0
np_img.shape (1, 32, 32)
np.min(np_img) 0.0
np.max(np_img) 0.99215686
np_label 4


/media/rene/ADV/lib/python3.6/site-packages/foolbox/models/pytorch.py:60: UserWarning: The PyTorch model is in training mode and therefore might not be deterministic. Call the eval() method to set it in evaluation mode if this is not intended.
  'The PyTorch model is in training mode and therefore might'


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[1, 1, 32, 32] to have 3 channels, but got 1 channels instead

In [2]:
result = pickle.load(open('/media/rene/data/adv_gen/MNIST/mnist_normal/models/results_ns10_pointwise_adv.pkl', "rb"))
display(result)

,adv_pred,attack_type,f_adv_pred,f_pred,orig_pred,path,trans_orig_pred,true_label
0,0.0,pointwise,6.0,2.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,0.0,9.0
1,0.0,pointwise,2.0,0.0,2.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,1.0,8.0
2,6.0,pointwise,0.0,0.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,0.0,8.0
3,2.0,pointwise,2.0,2.0,2.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,1.0,0.0
4,2.0,pointwise,4.0,4.0,4.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,1.0,0.0
5,3.0,pointwise,3.0,2.0,2.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,3.0,5.0
6,0.0,pointwise,0.0,0.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,0.0,1.0
7,0.0,pointwise,0.0,0.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,6.0,9.0
8,0.0,pointwise,0.0,0.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,0.0,9.0
9,0.0,pointwise,0.0,0.0,0.0,/media/rene/data/adv_gen/MNIST/mnist_normal/va...,0.0,1.0


In [ ]:
num_workers = 0 ## ??? what?
batch_size = 1

num_classes = 10
labels = list(range(num_classes))

with open(args.files_df_loc, 'rb') as f:
    files_df = pickle.load(f)
files_df['val'] = files_df['val'].sample(n=sample_num)

# Make generators:
if args.dataset == 'CIFAR10':
    dataloaders = make_generators_DF_cifar(files_df, batch_size, num_workers, size=IM_SIZE, 
                                            path_colname='path', adv_path_colname=None, return_loc=True)
elif args.dataset == 'MNIST':
    dataloaders = make_generators_DF_MNIST(files_df, batch_size, num_workers, size=IM_SIZE,
                                            path_colname='path', adv_path_colname=None, return_loc=True, normalize=False)

model = load_net(model_loc).to(device).eval()
gen_model = GenerativeCVAE(model=model, labels=labels, latent_size=16, device=device).eval()
fmodel = foolbox.models.PyTorchModel(gen_model, bounds=(0, 1), num_classes=num_classes, device=device) # no preprocessing since using data loader
attack  = get_attack(attack_type, fmodel)


for i, batch in enumerate(tqdm(dataloaders['val'])):
    problem_examples = 0
    img, label, file_loc = batch[0], batch[1], batch[2][0]

    # image, label, file_loc = batch[0].numpy(), int(batch[1].numpy()), batch[2][0]
    # image = load_image(file_loc) See if should use this with the foolbox preprocessign instead
    # np_img = np.expand_dims(np.squeeze(img.numpy()), 0)

    np_img = np.squeeze(img.numpy())
    np_label = int(label.numpy()[0])
    print('np_img.shape', np_img.shape)
    print('np_label', np_label)

    print('img.size()', img.size())
    print('torch.min(img)', torch.min(img))
    print('torch.max(img)', torch.max(img))


    adv_img = attack(np_img, np_label)

    attack = foolbox.attacks.AdditiveGaussianNoiseAttack(fmodel)
    adv_img = attack(img, label)


    print('type(adv_img)', type(adv_img))
    print('adv_img.shape', adv_img.shape)
    print('np.min(adv_img)', np.min(adv_img))
    print('np.max(adv_img)', np.max(adv_img))


    orig_pred = model(img)
    adv_pred = model(adv_img)

## Latent Descent Attack